In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
cwd = os.getcwd()

api_path = os.path.join(cwd, '..', 'tools', 'sampling_utils')
sys.path.append(api_path)

In [3]:
import numpy as np
import random
import sklearn

from matplotlib import pyplot as plt

import torch, torch.nn as nn
import torch.nn.functional as F


from sir_ais_sampling import (run_experiments_gaussians,
                              run_experiments_2_gaussians,
                              sir_correlated_dynamics,
                              sir_independent_dynamics)

from ebm_sampling import (citerais_mala_dynamics,
                          citerais_ula_dynamics, 
                          i_ais_z_dynamics,
                          i_ais_v_dynamics,
                          i_ais_b_dynamics)

from distributions import (Target, 
                           Gaussian_mixture, 
                           IndependentNormal,
                           init_independent_normal)

from torch.distributions import (MultivariateNormal, 
                                 Normal, 
                                 Independent, 
                                 Uniform)

from metrics import Evolution
from general_utils import DotDict, send_file_to_remote

In [4]:
from ebm_sampling import *

In [5]:
# def i_ais_v_dynamics(z, target, n_steps, grad_step, eps_scale, N, betas, rho):
#     z_sp = [z.clone().detach()]
#     batch_size, z_dim = z.shape[0], z.shape[1]
#     device = z.device

#     mh_transition = MH_Transition(z_dim, device)
#     acceptence_rate = [] #0.0

#     betas = np.array(betas)
#     betas_diff = torch.FloatTensor(betas[:-1] - betas[1:]) #n-1
    
#     scale_proposal = 1.0
#     proposal = init_independent_normal(scale_proposal, z_dim, device)

#     z_last = z
#     for step in tqdm(range(n_steps)):
#         if step == 0:
#             N_ = N
#         else:
#             N_ = N - 1

#         z = z_last.unsqueeze(1).repeat(1, N_, 1)

#         z = rho*z + ((1 - rho**2)**0.5) * proposal.sample([batch_size, N_])
        
#         z_batch = torch.transpose(z, 0, 1).reshape((batch_size*N_, z_dim)).detach().clone()
#         z_batch.requires_grad_(True)

#         # E = E.unsqueeze(1).repeat(1, N)
#         # grad = grad.unsqueeze(1).repeat(1, N, 1)
#         # grad = torch.transpose(grad, 0, 1).reshape((batch_size*N, z_dim)).data #detach().clone()

#         E, grad = grad_energy(z_batch, target)
#         E = E.reshape(N_, batch_size).T

#         z_backward = z_batch
#         z_backward.requires_grad_(True)
#         energy_backward = torch.zeros(batch_size, N_, len(betas)-1) # n-1
#         energy_backward[..., len(betas)-2] = E #.detach().clone()
        
#         E = E.T.reshape(-1)

#         accept = torch.zeros([len(betas)-2, N - 1])
#         for i, beta in enumerate(betas[::-1][1:-1]):  #betas[0] = 1, betas[n-1] = 0, betas[::-1][1:-1] - increasing, lenghts is n-2
#             j = len(betas) - i - 3
#             eps = eps_scale * proposal.sample([batch_size*N_])

#             z_backward_new = z_backward - beta * grad_step * grad + eps
#             z_backward, E, grad, mask = mh_transition.do_transition_step(z_backward, 
#                                                                          z_backward_new, 
#                                                                          E, 
#                                                                          grad, 
#                                                                          grad_step, 
#                                                                          eps_scale, 
#                                                                          target, 
#                                                                          beta=beta)
#             if step > 0:
#                 mask = mask.float().reshape(z.shape[:-1][::-1]).T.mean(0)
#                 accept[i] = mask #.sum()
#             E_ = E.reshape(z.shape[:-1][::-1]).T
#             energy_backward[..., j] = E_.detach().clone()
#         if step > 0:
#             acceptence_rate.append(accept)

#         z_backward = torch.transpose(z_backward.reshape(list(z.shape[:-1][::-1]) + [z.shape[-1]]), 0, 1)
#         #grad = torch.transpose(grad.reshape(list(z.shape[:-1][::-1]) + [z.shape[-1]]), 0, 1)
#         #E = E.reshape(z.shape[:-1][::-1]).T

#         if step > 0:
#             z = torch.cat([z, z_last.unsqueeze(1)], dim=1)
#             z_backward = torch.cat([z_backward, z_backward_last.unsqueeze(1)], dim=1)
#             energy_backward = torch.cat([energy_backward, energy_backward_last.unsqueeze(1)], dim=1)
#             #grad = torch.cat([grad, grad_last.unsqueeze(1)], dim=1)

#         #E = energy_backward[..., 0]

#         F_backward = (betas_diff[None, None, :] * energy_backward).sum(-1)
#         log_weights = -F_backward

#         max_logs = torch.max(log_weights, dim = 1)[0]
#         log_weights = log_weights - max_logs[:, None]
#         sum_weights = torch.logsumexp(log_weights, dim = 1)
#         log_weights = log_weights- sum_weights[:, None]
#         weights = log_weights.exp()
#         weights[weights != weights] = 0.
#         weights[weights.sum(1) == 0.] = 1.

#         indices = torch.multinomial(weights, 1).squeeze().tolist()

#         z_last = z[np.arange(batch_size), indices, :]
#         z_last = z_last.data
#         z_last.requires_grad_(True)

#         z_backward_last = z_backward[np.arange(batch_size), indices, :]
#         z_sp.append(z_backward_last.detach().clone())

#         #grad_last = grad[np.arange(batch_size), indices, :].data
#         #grad_last.requires_grad_(True)

#         energy_backward_last = energy_backward[np.arange(batch_size), indices, :]

#     #acceptence_rate = acceptence_rate/(batch_size*N)/len(betas[::-1][1:-1])/n_steps
#     #acceptence_rate = acceptence_rate/(n_steps - 1)
#     acceptence_rate = torch.stack(acceptence_rate, 0)
#     return z_sp, acceptence_rate

In [6]:
dim_arr = [2] + [30*(i + 1) for i in range(9)]  

loc_target = 3.0
loc_proposal = 0.0

var_proposal = 2.0
var_target = 1.0
scale_proposal = var_proposal ** 0.5
scale_target = var_target ** 0.5
num_points_in_chain = 100
strategy_mean = 'chain'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
batch_size = 100
n_steps = 300

In [7]:
dim = 2

target = init_independent_normal(scale_target, dim, 
                                       device, loc_target)
proposal = init_independent_normal(scale_proposal, dim, 
                                    device, loc_proposal)

In [8]:
N = 2

T = 50
betas = list(np.linspace(1., 0., T))
rhos = np.linspace(1, .7, T)

#betas = list(np.linspace(1., 0.85, 50)) + list(np.linspace(0.85, 0., 30))

grad_step = 0.1
eps_scale = (2*grad_step)**0.5

method_params = {'n_steps': n_steps, 
                 'N': N, 
                 'grad_step': grad_step, 
                 'eps_scale': eps_scale, 
                 'betas': betas,
                 'rho': 1.0,
                 'rhos': rhos}
random_seed = 42
method = 'citerais_mala' #'i_ais_v' 

In [9]:
start = proposal.sample([batch_size, len(betas)])

In [10]:
# history, acceptence = i_ais_v_dynamics(start, 
#                                         target.log_prob,
#                                         method_params['n_steps'], 
#                                         method_params['grad_step'], 
#                                         method_params['eps_scale'],
#                                         method_params['N'], 
#                                         method_params['betas'],
#                                         method_params['rho'])
                                            
history, acceptence = citerais_ula_dynamics(start, 
                                                target.log_prob,
                                                method_params['n_steps'], 
                                                method_params['grad_step'], 
                                                method_params['eps_scale'],
                                                method_params['N'], 
                                                method_params['betas'],
                                                method_params['rhos']
                                                )

100%|██████████| 300/300 [00:27<00:00, 11.09it/s]


In [11]:
# plt.plot(np.arange(78)+1, acceptence[120].mean(1))
# plt.savefig('../../accept_rate.png')

In [12]:
#acceptence[-1].mean(0)

In [13]:
#last_history = history[max(1, len(history) - num_points_in_chain - 1):]

In [14]:
last_history = history[-num_points_in_chain:]

In [15]:
len(last_history)

100

In [16]:
result = torch.stack(last_history, 0)

In [17]:
print((result[1:] != result[:-1]).float().min(-1)[0].mean(0)[0].item())

0.39393940567970276


In [18]:
result_np = result.cpu().numpy()

In [19]:
# over chain
result_var = np.var(result_np, axis = 0, ddof=1).mean(axis = 0).mean()
result_mean = np.mean(result_np, axis = 0).mean(axis = 0).mean()

# over batch
result_var = np.var(result_np, axis = 1, ddof=1).mean(axis = 0).mean()
result_mean = np.mean(result_np, axis = 1).mean(axis = 0).mean()

In [20]:
result_np.shape

(100, 100, 2)

In [21]:
result_mean, result_var

(1.612894, 0.5227851)

In [722]:
N = 10

T = 50
betas = np.linspace(1., 0., T)

grad_step = 0.5
eps_scale = (2*grad_step)**0.5

method_params = {'n_steps': n_steps, 
                 'N': N, 
                 'grad_step': grad_step, 
                 'eps_scale': eps_scale, 
                 'betas': betas,
                 'rho': 1.0}
random_seed = 42
method = 'i_ais_v' 
mode_init = 'proposal'
print_results = True

results_citerais_ula = run_experiments_gaussians(dim_arr,  
                                          scale_proposal, 
                                          scale_target, 
                                          loc_target,
                                          num_points_in_chain, 
                                          strategy_mean,
                                          device,
                                          batch_size,
                                          method_params,
                                          random_seed = random_seed,
                                          loc_proposal = loc_proposal,
                                          mode_init = mode_init,
                                          method = method,
                                          print_results = print_results) 

  0%|          | 2/1000 [00:00<01:01, 16.27it/s]------------------
mode = proposal
dim = 2
  5%|▌         | 50/1000 [00:02<00:45, 20.99it/s]


KeyboardInterrupt: 

In [674]:
strategy_mean

'chain'